# UTokyo Course Catalog Tutorial

I played with this dataset and found almost nothing zzz.

## Reading Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

df = pd.read_csv("../input/utokyosyllabus/all.csv", index_col=0)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
display(df.shape)
display(df.head(1))
display(df.describe())

In [ ]:
df = df.applymap(lambda x: 1 if x is True else 0 if x is False else x)
df.describe()

## KMeans

In [ ]:
X = df["単位数 他学部履修可 実務経験のある教員による授業科目 中間 期末 小テスト 演習 課題 レポート 発表 出席".split()]

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)

In [ ]:
kmeans.cluster_centers_

In [ ]:
kmeans.inertia_

In [ ]:
from sklearn.cluster import KMeans
from tqdm import tqdm
import matplotlib.pyplot as plt

cs = []
ns = [2 ** x for x in range(2, 7)]
for i in tqdm(ns):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X)
    cs.append(kmeans.inertia_)
plt.plot(ns, cs)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('CS')
plt.show()

## Grouping with faculty and departments

In [ ]:
df = pd.read_csv("../input/utokyosyllabus/all.csv", index_col=0)
df['scoring_sum'] = df["中間 期末 小テスト 演習 課題 レポート 発表 出席".split()].sum(axis=1)
df = df.rename(columns={"単位数":"credits", "実務経験のある教員による授業科目": "lecturer_has_practical_experience", "他学部履修可": "other_faculties_takable", "中間":"mid-exam", "期末":"final-exam", "小テスト":"quiz", "演習":"exercise", "課題":"homework", "レポート":"report", 
                  "発表": "presentation",  "出席": "attendance"})
g = df.groupby(['学部', '学科'])
display(g.mean())

In [ ]:
X = df["credits other_faculties_takable lecturer_has_practical_experience mid-exam final-exam quiz exercise homework report presentation attendance scoring_sum".split()]

## Kmeans with df.groupby().mean()

In [ ]:
X2 = g.mean()

In [ ]:
from sklearn.cluster import KMeans
from tqdm import tqdm
import matplotlib.pyplot as plt

cs = []
ns = range(1, 11)
for i in tqdm(ns):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X2)
    cs.append(kmeans.inertia_)
plt.plot(ns, cs)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('CS')
plt.show()

In [ ]:
import matplotlib.font_manager
print([f.name for f in matplotlib.font_manager.fontManager.ttflist])

It was unable for me to change the font.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

path = '../input/arial-font/arial.ttf'
prop = font_manager.FontProperties(fname=path)
mpl.rcParams['font.family'] = prop.get_name()

## Correlation matrix

In [ ]:
import seaborn as sns
sns.clustermap(X.corr(), metric='correlation', 
             standard_scale=1, linewidths=0) 

## Hierarchical Clustering

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage  
from scipy.spatial.distance import  pdist
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# Scaling
X = X.astype(float)
scaler = MinMaxScaler()  
solutions_set_scaled = scaler.fit_transform(X)
solutions_set_scaled = pd.DataFrame(X, columns = X.columns)
# transpose and convert solutions set to numpy
np_solutions_set = solutions_set_scaled.T.values
# calculate the distances
solutions_set_dist = pdist(np_solutions_set)
# hierarchical clusterization
linked = linkage(solutions_set_dist, 'ward')

In [ ]:
# dendrogram

fig = plt.figure(figsize=(8, 5))

dendrogram(linked, labels = solutions_set_scaled.columns)
plt.xticks(rotation=90)
plt.title('0.901-solutions clusters')
plt.show()

## I want to go to the department with the least attendance!

In [ ]:
with pd.option_context('display.max_rows', None,):
    count = g.count().iloc[:, 0]
    count.name = 'count'
    X3 = pd.concat([X2, count], axis=1)
    X3 = X3[X3["count"] > 15]
    display(X3.sort_values(by="attendance"))